In [3]:
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm
import scipy.ndimage
import cv2
import os
from matplotlib import pyplot as plt
import sys

# Get the home directory
home_directory = os.path.expanduser('~')

# Append the parent directory to the Python path
sys.path.append(os.path.join(home_directory, 'Ashish/UCAN', 'UCAN-PET-CT-image-data-handling-pipeline'))

from Utils import utils

def display_full(x):
    with pd.option_context("display.max_rows", None,
                           "display.max_columns", None,
                           "display.width", 20000,
                           "display.max_colwidth", None,
                           ):
        print(x)

In [4]:
CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/npr100169878450_SUV_CT/20130412/CT.nii.gz"

In [47]:
def get_proj_after_mask(img):
    
    '''

    Function to get 3D masks for CT scans to segment out the exact tissue type.

    '''

    pix_array=sitk.GetArrayFromImage(img)
    max_i, min_i=float(pix_array.max()),float(pix_array.min())

    #multiply= sitk.MultiplyImageFilter()
    #if hu_type == 'Bone' or hu_type == 'bone' or hu_type == 'B':
    bone_mask = sitk.BinaryThreshold(
    img, lowerThreshold=200, upperThreshold=max_i,insideValue=1, outsideValue=0
    )
    
    constrast_mask = sitk.BinaryThreshold(
    img, lowerThreshold=151, upperThreshold=199,insideValue=1, outsideValue=0
    )

    #bone_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        # path= img_n + r'_{0}_image.nii'.format(modality + '_' + type)
        # save_as_gz(op_img,path)
    #elif hu_type == 'Lean Tissue' or hu_type == 'lean' or hu_type == 'LT':
    lean_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-29, upperThreshold=150, insideValue=1, outsideValue=0
    )
    #lean_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))

    #elif hu_type == 'Adipose' or hu_type == 'adipose' or hu_type == 'AT':
    adipose_mask = sitk.BinaryThreshold(
    img, lowerThreshold=-199, upperThreshold=-30, insideValue=1, outsideValue=0
    )
    #adipose_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
        
    #elif hu_type == 'Air' or hu_type == 'A':
    air_mask = sitk.BinaryThreshold(
    img, lowerThreshold=min_i, upperThreshold=-191, insideValue=1, outsideValue=0
    )
    #air_mask = multiply.Execute(img,sitk.Cast(seg,img.GetPixelID()))
    
    return bone_mask, lean_mask, adipose_mask, air_mask, constrast_mask

In [44]:
CT_img = sitk.ReadImage(CT_path)

In [45]:
bone_mask, lean_mask, adipose_mask, air_mask, constrast_mask = get_proj_after_mask(CT_img)

In [10]:
lean_mask_arr = sitk.GetArrayFromImage(lean_mask)
np.unique(lean_mask_arr)

array([0, 1], dtype=uint8)

In [12]:
lean_mask_arr.sum()

2479840

In [15]:
np.unique(lean_mask_arr, return_counts=True)

(array([0, 1], dtype=uint8), array([16328992,  2479840]))

In [22]:
voxel_spacing = [2.0364201068878174, 2.0364201068878174, 3.0]

In [30]:
# calculate lean density of lean muscle # multiply with 1e-3 to covert mm3 to cm3 (ml)  # ml * 1e-3 litres
density = lean_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6

In [29]:
# calculate bone skeleton density
bone_mask_arr = sitk.GetArrayFromImage(bone_mask)
print(np.unique(bone_mask_arr))
print(bone_mask_arr.sum())
print(np.unique(bone_mask_arr, return_counts=True))
bone_mask_arr.sum() * voxel_spacing[0] * voxel_spacing[1] * voxel_spacing[2] * 1e-6

[0 1]
266094
(array([0, 1], dtype=uint8), array([18542738,   266094]))


3.3104809236183073

In [46]:
multiply= sitk.MultiplyImageFilter()
CT_constrast = multiply.Execute(CT_img,sitk.Cast(constrast_mask,CT_img.GetPixelID()))

CT_ptype = 'mean'
SUV_ptype = 'max'
angle = 90

raw_projections_path = "/home/ashish/Ashish/UCAN/Results/test_new_proj/"

utils.get_2D_projections(CT_constrast, 'CT', CT_ptype, angle, invert_intensity= True, img_n=raw_projections_path)
    

Finished generating 3 - mean intensity 2D projections from the CT volume image! 


In [52]:
resampled_directory_list = []
resampled_SUV_CT_path = "/home/ashish/Ashish/UCAN/Resampled_SUV_CT/"

for dirs, subdirs, files in os.walk(resampled_SUV_CT_path):
    for file in files:
        file_path = str(os.path.join(dirs, file))
        file_path = file_path.replace('\\','/')
        resampled_directory_list.append(file_path)

resampled_directory_df = pd.DataFrame(resampled_directory_list, columns=['directory'])
resampled_directory_df[['source_directory', 'patient_directory', 'scan_date', 'SUV_CT']] = resampled_directory_df['directory'].str.rsplit(pat='/', n=3, expand=True)
resampled_directory_df[['npr', 'extra']] = resampled_directory_df['patient_directory'].str.split(pat='_', n=1, expand=True)
resampled_directory_df.drop(columns=['directory','extra', 'SUV_CT'], inplace=True)
resampled_directory_df.drop_duplicates(inplace=True)

utils.display_full(resampled_directory_df.head(2))

# Generate the raw 2D projections
for index, row in resampled_directory_df.iterrows():

    CTnii_path = resampled_SUV_CT_path + str(row['npr']) + '_SUV_CT/' + str(row['scan_date']) + '/' + 'CT.nii.gz'
    
    CT_img =sitk.ReadImage(CTnii_path)

    bone_mask, lean_mask, adipose_mask, air_mask, constrast_mask = get_proj_after_mask(CT_img)

    multiply= sitk.MultiplyImageFilter()

    CT_constrast = multiply.Execute(CT_img,sitk.Cast(constrast_mask,CT_img.GetPixelID()))

    CT_bone = multiply.Execute(CT_img,sitk.Cast(bone_mask,CT_img.GetPixelID()))

    CT_ptype = 'mean'
    SUV_ptype = 'max'
    angle = 90

    raw_projections_path = "/home/ashish/Ashish/UCAN/Results/test_new_proj/"
    utils.get_2D_projections(CT_constrast, 'CT', SUV_ptype, angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_contrast_MaxIP_')
    utils.get_2D_projections(CT_bone, 'CT', SUV_ptype, angle, invert_intensity= True, img_n=raw_projections_path + str(row['npr']) + '_' + str(row['scan_date']) + '_CT_bone_MaxIP_')
    



                            source_directory       patient_directory scan_date              npr
0  /home/ashish/Ashish/UCAN/Resampled_SUV_CT  npr865130139443_SUV_CT  20210622  npr865130139443
2  /home/ashish/Ashish/UCAN/Resampled_SUV_CT  npr865130139443_SUV_CT  20211126  npr865130139443
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume image! 
Finished generating 3 - max intensity 2D projections from the CT volume 